

Architecture OverView:


        INPUT - CONV1 - RELU - CONV2 - RELU - MAXPOOL - FULLY_CONNECTED_LAYER - OUTPUT

In [ ]:
import numpy as np
import gzip
import time
import pickle

In [ ]:
## Hyperparameters
NUM_OUTPUT = 10
LEARNING_RATE = 0.03	#learning rate
IMG_WIDTH = 28
IMG_DEPTH = 1
FILTER_SIZE=5
NUM_FILT1 = 8
NUM_FILT2 = 8
BATCH_SIZE = 50
NUM_EPOCHS = 2	 # number of iterations
MU = 0.95
PICKLE_FILE = "trained.pickle"

In [ ]:
## Returns idexes of maximum value of the array
def nanargmax(a):
	idx = np.argmax(a, axis=None)
	multi_idx = np.unravel_index(idx, a.shape)
	if np.isnan(a[multi_idx]):
		nan_count = np.sum(np.isnan(a))
		# In numpy < 1.8 use idx = np.argsort(a, axis=None)[-nan_count-1]
		idx = np.argpartition(a, -nan_count-1, axis=None)[-nan_count-1]
		multi_idx = np.unravel_index(idx, a.shape)
	return multi_idx

def maxpool(X, f, s):
    l, w, h = X.shape
    pool = np.zeros((l, (w-f)//s + 1, (h-f)//s + 1))  # Use // for floor division
    for channel in range(0, l):
        row = 0
        for x in range(0, w-f+1, s):
            col = 0
            for y in range(0, h-f+1, s):
                pool[channel, row, col] = np.max(X[channel, x:x+f, y:y+f])
                col += 1
            row += 1
    return pool


def softmax_cost(out,y):
	eout = np.exp(out)#we dont have 128 a typo fixed
	probs = eout/sum(eout)
	
	p = sum(y*probs)
	cost = -np.log(p)	## (Only data loss. No regularised loss)
	return cost,probs	


## Returns gradient for all the paramaters in each iteration
def ConvNet(image, label, filt1, filt2, bias1, bias2, theta3, bias3):
	#####################################################################################################################
	#######################################  Feed forward to get all the layers  ########################################
	#####################################################################################################################

	## Calculating first Convolution layer
		
	## l - channel
	## w - size of square image
	## l1 - No. of filters in Conv1
	## l2 - No. of filters in Conv2
	## w1 - size of image after conv1
	## w2 - size of image after conv2
	(l, w, w) = image.shape		
	l1 = len(filt1)
	l2 = len(filt2)
	( _, f, f) = filt1[0].shape
	w1 = w-f+1
	w2 = w1-f+1
	
	conv1 = np.zeros((l1,w1,w1))
	conv2 = np.zeros((l2,w2,w2))

	for jj in range(0,l1):
		for x in range(0,w1):
			for y in range(0,w1):
				conv1[jj,x,y] = np.sum(image[:,x:x+f,y:y+f]*filt1[jj])+bias1[jj]
	conv1[conv1<=0] = 0 #relu activation

	## Calculating second Convolution layer
	for jj in range(0,l2):
		for x in range(0,w2):
			for y in range(0,w2):
				conv2[jj,x,y] = np.sum(conv1[:,x:x+f,y:y+f]*filt2[jj])+bias2[jj]
	conv2[conv2<=0] = 0 # relu activation

	## Pooled layer with 2*2 size and stride 2,2
	pooled_layer = maxpool(conv2, 2, 2)	

	fc1 = pooled_layer.reshape((int(w2/2)*int(w2/2)*l2,1))
	
	out = theta3.dot(fc1) + bias3	#10*1
	
	######################################################################################################################
	########################################  Using softmax function to get cost  ########################################
	######################################################################################################################
	cost, probs = softmax_cost(out, label.astype(float))
	if np.argmax(out)==np.argmax(label):
		acc=1
	else:
		acc=0
	#######################################################################################################################
	##########################  Backpropagation to get gradient	using chain rule of differentiation  ######################
	#######################################################################################################################
	dout = probs - label	#	dL/dout
	
	dtheta3 = dout.dot(fc1.T) 		##	dL/dtheta3

	dbias3 = sum(dout.T).T.reshape((10,1))		##	dbias3	

	dfc1 = theta3.T.dot(dout)		##	dL/dfc1

	dpool = dfc1.T.reshape((l2, int(w2/2),int(w2/2)))

	dconv2 = np.zeros((l2, w2, w2))
	
	for jj in range(0,l2):
		i=0
		while(i<w2):
			j=0
			while(j<w2):
				(a,b) = nanargmax(conv2[jj,i:i+2,j:j+2]) ## Getting indexes of maximum value in the array
				dconv2[jj,i+a,j+b] = dpool[jj,int(i/2),int(j/2)]
				j+=2
			i+=2
	
	dconv2[conv2<=0]=0

	dconv1 = np.zeros((l1, w1, w1))
	dfilt2 = {}
	dbias2 = {}
	for xx in range(0,l2):
		dfilt2[xx] = np.zeros((l1,f,f))
		dbias2[xx] = 0

	dfilt1 = {}
	dbias1 = {}
	for xx in range(0,l1):
		dfilt1[xx] = np.zeros((l,f,f))
		dbias1[xx] = 0

	for jj in range(0,l2):
		for x in range(0,w2):
			for y in range(0,w2):
				dfilt2[jj]+=dconv2[jj,x,y]*conv1[:,x:x+f,y:y+f]
				dconv1[:,x:x+f,y:y+f]+=dconv2[jj,x,y]*filt2[jj]
		dbias2[jj] = np.sum(dconv2[jj])
	dconv1[conv1<=0]=0
	for jj in range(0,l1):
		for x in range(0,w1):
			for y in range(0,w1):
				dfilt1[jj] += dconv1[jj, x, y] * image[:, x:x+f, y:y+f].astype(float)

		dbias1[jj] = np.sum(dconv1[jj])

	
	return [dfilt1, dfilt2, dbias1, dbias2, dtheta3, dbias3, cost, acc]


def initialize_param(f, l):
	return 0.01*np.random.rand(l, f, f)

def initialize_theta(NUM_OUTPUT, l_in):
	return 0.01*np.random.rand(NUM_OUTPUT, l_in)

def initialise_param_lecun_normal(FILTER_SIZE, IMG_DEPTH, scale=1.0, distribution='normal'):
	
    if scale <= 0.:
            raise ValueError('`scale` must be a positive float. Got:', scale)

    distribution = distribution.lower()
    if distribution not in {'normal'}:
        raise ValueError('Invalid `distribution` argument: '
                             'expected one of {"normal", "uniform"} '
                             'but got', distribution)

    scale = scale
    distribution = distribution
    fan_in = FILTER_SIZE*FILTER_SIZE*IMG_DEPTH
    scale = scale
    stddev = scale * np.sqrt(1./fan_in)
    shape = (IMG_DEPTH,FILTER_SIZE,FILTER_SIZE)
    return np.random.normal(loc = 0,scale = stddev,size = shape)

## Returns all the trained parameters
def momentumGradDescent(batch, LEARNING_RATE, w, l, MU, filt1, filt2, bias1, bias2, theta3, bias3, cost, acc):
	#	Momentum Gradient Update
	# MU=0.5	
	X = batch[:,0:-1]
	X = X.reshape(len(batch), l, w, w)
	y = batch[:,-1]

	n_correct=0
	cost_ = 0
	batch_size = len(batch)
	dfilt2 = {}
	dfilt1 = {}
	dbias2 = {}
	dbias1 = {}
	v1 = {}
	v2 = {}
	bv1 = {}
	bv2 = {}
	for k in range(0,len(filt2)):
		dfilt2[k] = np.zeros(filt2[0].shape)
		dbias2[k] = 0
		v2[k] = np.zeros(filt2[0].shape)
		bv2[k] = 0
	for k in range(0,len(filt1)):
		dfilt1[k] = np.zeros(filt1[0].shape)
		dbias1[k] = 0
		v1[k] = np.zeros(filt1[0].shape)
		bv1[k] = 0
	dtheta3 = np.zeros(theta3.shape)
	dbias3 = np.zeros(bias3.shape)
	v3 = np.zeros(theta3.shape)
	bv3 = np.zeros(bias3.shape)



	for i in range(0,batch_size):
		
		image = X[i]

		label = np.zeros((theta3.shape[0],1))
		label[int(y[i]),0] = 1
		
		## Fetching gradient for the current parameters
		[dfilt1_, dfilt2_, dbias1_, dbias2_, dtheta3_, dbias3_, curr_cost, acc_] = ConvNet(image, label, filt1, filt2, bias1, bias2, theta3, bias3)
		for j in range(0,len(filt2)):
			dfilt2[j]+=dfilt2_[j]
			dbias2[j]+=dbias2_[j]
		for j in range(0,len(filt1)):
			dfilt1[j]+=dfilt1_[j]
			dbias1[j]+=dbias1_[j]
		dtheta3+=dtheta3_
		dbias3+=dbias3_

		cost_+=curr_cost
		n_correct+=acc_

	for j in range(0,len(filt1)):
		v1[j] = MU*v1[j] -LEARNING_RATE*dfilt1[j]/batch_size
		filt1[j] += v1[j]
		# filt1[j] -= LEARNING_RATE*dfilt1[j]/batch_size
		bv1[j] = MU*bv1[j] -LEARNING_RATE*dbias1[j]/batch_size
		bias1[j] += bv1[j]
	for j in range(0,len(filt2)):
		v2[j] = MU*v2[j] -LEARNING_RATE*dfilt2[j]/batch_size
		filt2[j] += v2[j]
		# filt2[j] += -LEARNING_RATE*dfilt2[j]/batch_size
		bv2[j] = MU*bv2[j] -LEARNING_RATE*dbias2[j]/batch_size
		bias2[j] += bv2[j]
	v3 = MU*v3 - LEARNING_RATE*dtheta3/batch_size
	theta3 += v3
	# theta3 += -LEARNING_RATE*dtheta3/batch_size
	bv3 = MU*bv3 -LEARNING_RATE*dbias3/batch_size
	bias3 += bv3

	cost_ = cost_/batch_size
	cost.append(cost_)
	accuracy = float(n_correct)/batch_size
	acc.append(accuracy)

	return [filt1, filt2, bias1, bias2, theta3, bias3, cost, acc]

## Predict class of each row of matrix X
def predict(image, filt1, filt2, bias1, bias2, theta3, bias3):
	
	## l - channel
	## w - size of square image
	## l1 - No. of filters in Conv1
	## l2 - No. of filters in Conv2
	## w1 - size of image after conv1
	## w2 - size of image after conv2

	(l,w,w)=image.shape
	(l1,f,f) = filt2[0].shape
	l2 = len(filt2)
	w1 = w-f+1
	w2 = w1-f+1
	conv1 = np.zeros((l1,w1,w1))
	conv2 = np.zeros((l2,w2,w2))
	for jj in range(0,l1):
		for x in range(0,w1):
			for y in range(0,w1):
				conv1[jj,x,y] = np.sum(image[:,x:x+f,y:y+f]*filt1[jj])+bias1[jj]
	conv1[conv1<=0] = 0 #relu activation
	## Calculating second Convolution layer
	for jj in range(0,l2):
		for x in range(0,w2):
			for y in range(0,w2):
				conv2[jj,x,y] = np.sum(conv1[:,x:x+f,y:y+f]*filt2[jj])+bias2[jj]
	conv2[conv2<=0] = 0 # relu activation
	## Pooled layer with 2*2 size and stride 2,2
	pooled_layer = maxpool(conv2, 2, 2)	
	fc1 = pooled_layer.reshape((int(w2/2)*int(w2/2)*l2, 1))
	out = theta3.dot(fc1) + bias3	#10*1
	eout = np.exp(out, dtype=np.float)
	probs = eout/sum(eout)
	# probs = 1/(1+np.exp(-out))

	# print out
	# print np.argmax(out), np.max(out)
	return np.argmax(probs), np.max(probs)


def extract_data(filename, num_images, IMAGE_WIDTH):
	"""Extract the images into a 4D tensor [image index, y, x, channels].
	Values are rescaled from [0, 255] down to [-0.5, 0.5].
	"""
	print('Extracting', filename)
	with gzip.open(filename) as bytestream:
		bytestream.read(16)
		buf = bytestream.read(IMAGE_WIDTH * IMAGE_WIDTH * num_images)
		data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
		data = data.reshape(num_images, IMAGE_WIDTH*IMAGE_WIDTH)
		return data

def extract_labels(filename, num_images):
	"""Extract the labels into a vector of int64 label IDs."""
	print('Extracting', filename)
	with gzip.open(filename) as bytestream:
		bytestream.read(8)
		buf = bytestream.read(1 * num_images)
		labels = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
	return labels

In [ ]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784')

X_data, y_data = mnist.data, mnist.target



In [ ]:
## load your data
X = X_data.to_numpy()
y = y_data.to_numpy().reshape(70000, 1)

# normalize images
X-=int(np.mean(X))
X/=int(np.std(X))

train_data = np.hstack((X,y))

np.random.shuffle(train_data)

NUM_IMAGES = train_data.shape[0]

filt1 = {}
bias1 = {}

filt2 = {}
bias2 = {}

for i in range(0,NUM_FILT1):
    filt1[i] = initialise_param_lecun_normal(FILTER_SIZE, IMG_DEPTH, scale=1.0)
    bias1[i] = 0

for i in range(0,NUM_FILT2):
    filt2[i] = initialise_param_lecun_normal(FILTER_SIZE, NUM_FILT1, scale=1.0)
    bias2[i] = 0

w1 = IMG_WIDTH-FILTER_SIZE+1
w2 = w1-FILTER_SIZE+1
theta3 = initalize_theta(NUM_OUTPUT, (w2/2)*(w2/2)*NUM_FILT2)

bias3 = np.zeros((NUM_OUTPUT, 1))
cost = []
acc = []

print("Learning Rate:"+str(LEARNING_RATE)+", Batch Size:"+str(BATCH_SIZE))

## Start training from here

In [ ]:
for iteration in range(0,NUM_EPOCHS):
    
    np.random.shuffle(train_data)

    batches = [train_data[k:k+BATCH_SIZE] for k in range(0,NUM_IMAGES, BATCH_SIZE)]

    x = 0
    for j,batch in enumerate(batches):
        stime = time.time()
        output = momentumGradDescent(batch, LEARNING_RATE, IMG_WIDTH, IMG_DEPTH, MU, filt1, filt2, bias1, bias2, theta3, bias3, cost, acc)
        [filt1, filt2, bias1, bias2, theta3, bias3, cost, acc] = output

        epoch_acc = round(np.sum(acc[int(iteration*NUM_IMAGES/BATCH_SIZE):][x+1]), 2)

        per = float(x+1)/len(batches)*100

        # print("Epoch:"+str(round(per,2))+"% Of "+str(iteration+1)+"/"+str(NUM_EPOCHS)+", Cost:"+str(cost[-1])+", B.Acc:"+str(acc[-1]*100)+", E.Acc:"+str(epoch_acc))

        ftime = time.time()

        print(f"total time taken for epoch {iteration+1} and batch {j+1}: ",(ftime - stime))

        x += 1

with open(PICKLE_FILE, 'wb') as file:
	pickle.dump(output, file)

In [ ]:
y.shape